In [0]:
from pyspark.sql import SparkSession # Puerta de entrada para trabajar con spark <-- SIEMPRE DEBEMOS IMPORTAR LA LLAVE MAESTRA QUE INICIA TODO.
from pyspark.sql.functions import *  # Funciones propias del módulo SQL de Spark, para trabajar sobre Dataframes.
from pyspark.sql.types import *
spark = SparkSession.builder.appName("RangoInterquartilApacheSparkDatabricks").getOrCreate() 
"""
^          ^__________^        ^_________^                               ^
|                |                   |                                   | 
Variable   Constructor de Sesión   Nombre Aplicación       Evita conflicto del SparkSession"""


### DATAFRAMES (Versión Databricks Free Edition)

#### Link de [Databrick Free Edition](https://dbc-89f542f8-2df6.cloud.databricks.com/?o=758509963140561)

##### VALORES ATÍPICOS Y NO ATÍPICOS (RANGO INTERQUARTIL)

In [0]:
# 🔍 Análisis de valores atípicos usando el rango intercuartil (IQR)
# Dataset utilizado: 🐧 Penguins (cargado previamente como Data Table en Unity Catalog)
# ⚠️ Próximamente explicaré cómo cargar datasets al Unity Catalog en Databricks Free Edition...

df_penguins = spark.sql("SELECT * FROM workspace.exercises.penguins") #📊LENGUAJE SQL¿?....
# df_penguins.show()

# ESTADÍSTICA DESCRIPTIVA BÁSICA DE UN DATASET (PENGUINS)
# df_penguins.describe().show()

# Rango Interquartil (IQR)
# Permite hallar valores atípicos dentro de nuestro dataset.
# 💡Rango interquartil aplicado a la columna body_mass_g
quartiles_body_mass_g = df_penguins.agg(
expr('percentile(body_mass_g,array(0.25))')[0].alias("q1"), # 💡Importante: expr() viene integrado dentro
expr('percentile(body_mass_g,array(0.50))')[0].alias("q2"), #    de las funciones sql en pyspark.
expr('percentile(body_mass_g,array(0.75))')[0].alias("q3"), # 💡 Cada cuartil es una porción del dataset completo
expr('percentile(body_mass_g,array(1.00))')[0].alias("q4")  #    (0.25,0.50,0.75,1) 
)
# quartiles_body_mass_g.show()

q1 = quartiles_body_mass_g[["q1"]].first() # Quartil 1 ➡️ método .first() trae el primer valor de la columna
## print(q1.q1) # Accede al valor de la columna
q3 = quartiles_body_mass_g[["q3"]].first() # Quartil 3 ➡️ método .first() trae el primer valor de la columna
## print(q3.q3)# Accede al valor de la columna
rango_iqr = q3.q3-q1.q1 # Cálculo de rango intercuartil
## print(rango_iqr)
lower_bound_body = q1.q1 - 1.5 *rango_iqr # Límite superior
upper_bound_body = q3.q3 + 1.5 * rango_iqr # Límite inferior
## print(lower_bound_body)
## print(upper_bound_body)
df_penguins_atipicos = df_penguins.filter(  
    (col("body_mass_g")<lower_bound_body) |
    (col("body_mass_g")>upper_bound_body)
)# ⬅️ Filtramos los valores atípicos (fuera de los umbrales calculados en los límites superior e inferior)
## df_penguins_atipicos.show()
df_penguins_normales = df_penguins.filter(
    (col("body_mass_g")>=lower_bound_body) &
    (col("body_mass_g")<=upper_bound_body)
)# ⬅️ Filtramos los valores limpios(dentro de los umbrales calculados en los límites superior e inferior)
df_penguins_normales.show() # Dataset limpio